# Import Libraries

In [2]:
import gc
import warnings
warnings.filterwarnings('ignore')
import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
import itertools
from dotenv import dotenv_values

In [3]:
config = dotenv_values('../.env')

In [9]:
def read_preprocess_data():
    train = pd.read_parquet(config["INT_TRAIN_PARQUET"])
    features = train.drop(['customer_ID', 'S_2'], axis = 1).columns.to_list()
    cat_features = [
        "B_30",
        "B_38",
        "D_114",
        "D_116",
        "D_117",
        "D_120",
        "D_126",
        "D_63",
        "D_64",
        "D_66",
        "D_68",
    ]
    num_features = [col for col in features if col not in cat_features]
    
    # Train FE
    print('Starting train feature extraction')
    train_num_agg = train.groupby("customer_ID")[num_features].agg(['first', 'mean', 'std', 'min', 'max', 'last'])
    train_num_agg.columns = ['_'.join(x) for x in train_num_agg.columns]
    train_num_agg.reset_index(inplace = True)
    
    # Lag Features
    for col in train_num_agg:
        if 'last' in col and col.replace('last', 'first') in train_num_agg:
            train_num_agg[col + '_lag_sub'] = train_num_agg[col] - train_num_agg[col.replace('last', 'first')]
            train_num_agg[col + '_lag_div'] = train_num_agg[col] / train_num_agg[col.replace('last', 'first')]

    train_cat_agg = train.groupby("customer_ID")[cat_features].agg(['count', 'first', 'last', 'nunique'])
    train_cat_agg.columns = ['_'.join(x) for x in train_cat_agg.columns]
    train_cat_agg.reset_index(inplace = True)
    
    train_labels = pd.read_csv(config["TRAIN_LABELS_PATH"])
    train = train_num_agg.merge(train_cat_agg, how = 'inner', on = 'customer_ID').merge(train_labels, how = 'inner', on = 'customer_ID')
    print('Train shape: ', train.shape)    
    del train_num_agg, train_cat_agg        
    gc.collect()
    
    # Test FE
    test = pd.read_parquet(config["INT_TEST_PARQUET"])
    print('Starting test feature extraction')
    test_num_agg = test.groupby("customer_ID")[num_features].agg(['first', 'mean', 'std', 'min', 'max', 'last'])
    test_num_agg.columns = ['_'.join(x) for x in test_num_agg.columns]
    test_num_agg.reset_index(inplace = True)
    
     # Lag Features
    for col in test_num_agg:
        if 'last' in col and col.replace('last', 'first') in test_num_agg:
            test_num_agg[col + '_lag_sub'] = test_num_agg[col] - test_num_agg[col.replace('last', 'first')]
            test_num_agg[col + '_lag_div'] = test_num_agg[col] / test_num_agg[col.replace('last', 'first')]

    test_cat_agg = test.groupby("customer_ID")[cat_features].agg(['count', 'first', 'last', 'nunique'])
    test_cat_agg.columns = ['_'.join(x) for x in test_cat_agg.columns]
    test_cat_agg.reset_index(inplace = True)
    
    test = test_num_agg.merge(test_cat_agg, how = 'inner', on = 'customer_ID')
    print('Test shape: ', test.shape)
    del test_num_agg, test_cat_agg
    gc.collect()
    
    
    # Save files to disk
    train.to_parquet(config["ENGINEERED_DATA"] + 'train_fe_plus_plus.parquet')
    test.to_parquet(config["ENGINEERED_DATA"] + 'test_fe_plus_plus.parquet')

In [10]:
read_preprocess_data()

Starting train feature extraction
Train shape:  (458913, 1462)
Starting test feature extraction
Test shape:  (924621, 1461)


In [12]:
train = pd.read_parquet(config["ENGINEERED_DATA"] + 'train_fe_plus_plus.parquet')

In [13]:
train.head()

,customer_ID,P_2_first,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_first,D_39_mean,D_39_std,D_39_min,D_39_max,D_39_last,B_1_first,B_1_mean,B_1_std,B_1_min,B_1_max,B_1_last,B_2_first,B_2_mean,B_2_std,B_2_min,B_2_max,B_2_last,R_1_first,R_1_mean,R_1_std,R_1_min,R_1_max,R_1_last,S_3_first,S_3_mean,S_3_std,S_3_min,S_3_max,S_3_last,D_41_first,D_41_mean,D_41_std,D_41_min,D_41_max,D_41_last,B_3_first,B_3_mean,B_3_std,B_3_min,B_3_max,B_3_last,D_42_first,D_42_mean,D_42_std,D_42_min,D_42_max,D_42_last,D_43_first,D_43_mean,D_43_std,D_43_min,D_43_max,D_43_last,D_44_first,D_44_mean,D_44_std,D_44_min,D_44_max,D_44_last,B_4_first,B_4_mean,B_4_std,B_4_min,B_4_max,B_4_last,D_45_first,D_45_mean,D_45_std,D_45_min,D_45_max,D_45_last,B_5_first,B_5_mean,B_5_std,B_5_min,B_5_max,B_5_last,R_2_first,R_2_mean,R_2_std,R_2_min,R_2_max,R_2_last,D_46_first,D_46_mean,D_46_std,D_46_min,D_46_max,D_46_last,D_47_first,D_47_mean,D_47_std,D_47_min,D_47_max,D_47_last,D_48_first,D_48_mean,D_48_std,D_48_min,D_48_max,D_48_last,D_49_first,D_49_mean,D_49_std,D_49_min,D_49_max,D_49_last,B_6_first,B_6_mean,B_6_std,B_6_min,B_6_max,B_6_last,B_7_first,B_7_mean,B_7_std,B_7_min,B_7_max,B_7_last,B_8_first,B_8_mean,B_8_std,B_8_min,B_8_max,B_8_last,D_50_first,D_50_mean,D_50_std,D_50_min,D_50_max,D_50_last,D_51_first,D_51_mean,D_51_std,D_51_min,D_51_max,D_51_last,B_9_first,B_9_mean,B_9_std,B_9_min,B_9_max,B_9_last,R_3_first,R_3_mean,R_3_std,R_3_min,R_3_max,R_3_last,D_52_first,D_52_mean,D_52_std,D_52_min,D_52_max,D_52_last,P_3_first,P_3_mean,P_3_std,P_3_min,P_3_max,P_3_last,B_10_first,B_10_mean,B_10_std,B_10_min,B_10_max,B_10_last,D_53_first,D_53_mean,D_53_std,D_53_min,D_53_max,D_53_last,S_5_first,S_5_mean,S_5_std,S_5_min,S_5_max,S_5_last,B_11_first,B_11_mean,B_11_std,B_11_min,B_11_max,B_11_last,S_6_first,S_6_mean,S_6_std,S_6_min,S_6_max,S_6_last,D_54_first,D_54_mean,D_54_std,D_54_min,D_54_max,D_54_last,R_4_first,R_4_mean,R_4_std,R_4_min,R_4_max,R_4_last,S_7_first,S_7_mean,S_7_std,S_7_min,S_7_max,S_7_last,B_12_first,B_12_mean,B_12_std,B_12_min,B_12_max,B_12_last,S_8_first,S_8_mean,S_8_std,S_8_min,S_8_max,S_8_last,D_55_first,D_55_mean,D_55_std,D_55_min,D_55_max,D_55_last,D_56_first,D_56_mean,D_56_std,D_56_min,D_56_max,D_56_last,B_13_first,B_13_mean,B_13_std,B_13_min,B_13_max,B_13_last,R_5_first,R_5_mean,R_5_std,...,R_7_last_lag_div,D_77_last_lag_sub,D_77_last_lag_div,B_25_last_lag_sub,B_25_last_lag_div,B_26_last_lag_sub,B_26_last_lag_div,D_78_last_lag_sub,D_78_last_lag_div,D_79_last_lag_sub,D_79_last_lag_div,R_8_last_lag_sub,R_8_last_lag_div,R_9_last_lag_sub,R_9_last_lag_div,S_16_last_lag_sub,S_16_last_lag_div,D_80_last_lag_sub,D_80_last_lag_div,R_10_last_lag_sub,R_10_last_lag_div,R_11_last_lag_sub,R_11_last_lag_div,B_27_last_lag_sub,B_27_last_lag_div,D_81_last_lag_sub,D_81_last_lag_div,D_82_last_lag_sub,D_82_last_lag_div,S_17_last_lag_sub,S_17_last_lag_div,R_12_last_lag_sub,R_12_last_lag_div,B_28_last_lag_sub,B_28_last_lag_div,R_13_last_lag_sub,R_13_last_lag_div,D_83_last_lag_sub,D_83_last_lag_div,R_14_last_lag_sub,R_14_last_lag_div,R_15_last_lag_sub,R_15_last_lag_div,D_84_last_lag_sub,D_84_last_lag_div,R_16_last_lag_sub,R_16_last_lag_div,B_29_last_lag_sub,B_29_last_lag_div,S_18_last_lag_sub,S_18_last_lag_div,D_86_last_lag_sub,D_86_last_lag_div,D_87_last_lag_sub,D_87_last_lag_div,R_17_last_lag_sub,R_17_last_lag_div,R_18_last_lag_sub,R_18_last_lag_div,D_88_last_lag_sub,D_88_last_lag_div,B_31_last_lag_sub,B_31_last_lag_div,S_19_last_lag_sub,S_19_last_lag_div,R_19_last_lag_sub,R_19_last_lag_div,B_32_last_lag_sub,B_32_last_lag_div,S_20_last_lag_sub,S_20_last_lag_div,R_20_last_lag_sub,R_20_last_lag_div,R_21_last_lag_sub,R_21_last_lag_div,B_33_last_lag_sub,B_33_last_lag_div,D_89_last_lag_sub,D_89_last_lag_div,R_22_last_lag_sub,R_22_last_lag_div,R_23_last_lag_sub,R_23_last_lag_div,D_91_last_lag_sub,D_91_last_lag_div,D_92_last_lag_sub,D_92_last_lag_div,D_93_last_lag_sub,D_93_last_lag_div,D_94_last_lag_sub,D_94_last_lag_div,R_24_last_lag_sub,R_24_last_lag_div,R_25_last_lag_sub,R_25_last_l

In [14]:
train.isnull().sum()

customer_ID        0
P_2_first       2434
P_2_mean        2434
P_2_std         7829
P_2_min         2434
                ... 
D_68_count         0
D_68_first         0
D_68_last          0
D_68_nunique       0
target             0
Length: 1462, dtype: int64